In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, make_scorer
import joblib
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1, 16, 25"

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == "cuda":
    print('GPU available')
else:
    print('GPU not available')
device

GPU available


device(type='cuda')

In [3]:
task2_test_set = "610_ps4_test/testingT2HAR/har_test.csv"

# task2_test_set = "610_ps4_testing/testingT2HAR/har_test.csv"

In [14]:
# CONVERT DEV FILE INTO A DATAFRAME
task2_df = pd.read_csv(task2_test_set, header=None)
column_names = task2_df.columns
label = column_names[-1]

# Everything except the last element are features
X = task2_df.drop(columns=[label])
y = task2_df[label] 

# Make sure all columns are numeric
X = X.apply(pd.to_numeric)

# change the data structure from pandas to numpy 
X_numpy_arr = X.to_numpy()
y_numpy_arr = y.to_numpy()

In [5]:
# print(X)
important_features = [63, 6, 64, 46, 15, 68, 101, 132, 110, 7, 24, 94, 9, 151, 161, 36, 98, 119, 12, 65, 79, 43, 138, 178, 141, 30, 134, 148, 66, 133, 34, 19, 77, 169]

In [6]:
# Get the important feature columns only
X_important_features = X[important_features]

In [7]:
# change the data structure from pandas to numpy 
X_numpy_arr = X_important_features.to_numpy()
y_test_numpy_arr = y.to_numpy()

In [8]:
# Load the saved model
model_name = 'task2_randomforest_smote_balanced_rfecv_model.joblib'
rf_loaded = joblib.load(model_name)

In [9]:
 # PREDICT
y_pred = rf_loaded.predict(X_numpy_arr)

# CALCULATE accuracy, macro f1 score, and f1 score for fraud
model_accuracy = accuracy_score(y_test_numpy_arr, y_pred)
macro_avg_f1_score = f1_score(y_test_numpy_arr, y_pred, average='macro')

print("Accuracy:", model_accuracy)
print("Macro average F1 score:", macro_avg_f1_score)

Accuracy: 0.7962962962962963
Macro average F1 score: 0.6468366756392473


In [12]:
# accuracy = accuracy_score(y, y_pred)
# precision = precision_score(y, y_pred)
# recall = recall_score(y, y_pred)
# f1 = f1_score(y, y_pred)

# print(f'Accuracy: {accuracy}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')
# print(f'F1 Score: {f1}')

cm = confusion_matrix(y_test_numpy_arr, y_pred)
print(cm)

print(classification_report(y_test_numpy_arr, y_pred))

[[ 43   1   0   0   0   0   1   1   0   2   0]
 [  0  59   1   4   0   0   1   1   0   1   1]
 [  0   0   2   0   0   0   0   0   0   0   0]
 [  0   2   1  15   0   0   0   0   0   0   3]
 [  0   0   0   0   1   0   1   0   0   0   1]
 [  0   0   0   0   0 120   1   2   0   0   0]
 [  2   2   0   0   1   0  14   2   1   0   0]
 [  0  16   0   5   0   2   2  71   2   2   3]
 [  0   0   0   0   0   0   0   0   9   0   1]
 [  3   3   0   0   0   0   2   0   0   2   5]
 [  0   0   0   0   0   0   0   2   2   5   8]]
                   precision    recall  f1-score   support

          Cycling       0.90      0.90      0.90        48
         Football       0.71      0.87      0.78        68
          Jogging       0.50      1.00      0.67         2
         JumpRope       0.62      0.71      0.67        21
          Pushups       0.50      0.33      0.40         3
          Sitting       0.98      0.98      0.98       123
         Swimming       0.64      0.64      0.64        22
         

In [29]:
output_df = task2_df.drop(columns=[label])
new_last_index = len(output_df.loc[0])
output_df[new_last_index] = y_pred
output_df.to_csv('task2_test_output.csv', header=False, index=False) 